# Visualisation et traitement statistique des données chimiques

In [ ]:
import ipywidgets as widgets
import io
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Inspiré de https://fxjollois.github.io/cours-2019-2020/lp-iot--python-ds/seance2-ACP-classif.html


## Upload du fichier excel de données

In [ ]:
uploader_dataset = widgets.FileUpload(description="AGLAE PCA dataset")
display(uploader_dataset)

In [ ]:
reliure_complet = pd.read_excel(io.BytesIO(uploader_dataset.value[0].content), sheet_name = "translucid transparent", header = 0, index_col=None)
print("Aperçu du DataFrame:")
reliure_complet.head(5)

## Standardisation des données pour PCA

On sélectionne uniquement les colonnes numériques correspondant aux éléments chimiques

In [ ]:
reliure_num = reliure_complet.drop(columns = ["Localisation","Zone mesurée","Couleur","Type de verre"])
print("\nColonnes des éléments chimiques utilisées pour la PCA:")
reliure_num.head(5)

### Normalisation du dataset à 100% si on a enlevé des éléments chimiques

In [ ]:
reliure_num_normalise = reliure_num.div(reliure_num.sum(axis=1), axis=0) * 100
reliure_num_normalise.head(5)

On effectue la standardisation soit le centrage (soustration de la moyenne de chaque variable) puis la réduction (division de chaque variable par son écart-type) des données avec la fonction StandardScaler(). Le but est d'obtenir une moyenne nulle et un écart-type égal à 1 pour chaque variable. Cela garantit que chaque variable contribue de manière équitable à l'analyse sans qu'une variable à forte variance ne domine les résultats de la PCA.

In [ ]:
scaler = StandardScaler()
reliure_num_norm = scaler.fit_transform(reliure_num_normalise)

## Calcul de la PCA

In [ ]:
pca = PCA()
pca.fit(reliure_num_norm)
#resultat_pca = pca.fit_transform(colonnes_elements_standardisees)

print(pca.explained_variance_)
print(pca.explained_variance_ratio_)
print(pca.n_components_)

### Variance expliquée 
On fait un tableau récapitulatif avec les variances expliquées et le ratio de variance expliquée par dimension.

In [ ]:
eig = pd.DataFrame(
    {
        "Dimension" : ["Dim" + str(x + 1) for x in range(pca.n_components_)], 
        "Variance expliquée" : pca.explained_variance_,
        "% variance expliquée" : np.round(pca.explained_variance_ratio_ * 100),
        "% cum. var. expliquée" : np.round(np.cumsum(pca.explained_variance_ratio_) * 100)
    }
)

eig

On représente graphiquement les proportions de variance expliquée

In [ ]:
# On choisit un subset du tableau avec des valeurs non nulles
eig_subset = eig.iloc[:16] 
ax = eig_subset.plot.bar(x="Dimension", y="% variance expliquée", color="blue")
# Parcourir chaque barre (patch) pour annoter sa valeur
for p in ax.patches:
    # p.get_height() correspond à la hauteur de la barre (la valeur y)
    ax.annotate(f'{p.get_height():.0f}%', 
                (p.get_x() + p.get_width() / 2, p.get_height()), 
                ha='center', va='bottom', fontsize=12)
plt.show()


## Représentation des individus
Nous allons maintenant calculer les coordonnées des individus sur les dimensions avec la fonction transform() de l'objet pca

In [ ]:
reliure_pca = pca.transform(reliure_num_norm)

# On crée un dataframe avec le nombre de colonnes = le nombre de composantes obtenues par la PCA
reliure_pca_df = pd.DataFrame(reliure_pca, columns = [f"Dim{i+1}" for i in range(reliure_pca.shape[1])])
reliure_pca_df["Localisation"] = reliure_complet["Localisation"]
reliure_pca_df["Couleur"] = reliure_complet["Couleur"]

reliure_pca_df.head(10)


Il est maintenant possible de représenter les données sur le premier plan factoriel. Dans ce graphique, il est important de noter le pourcentage de variance expliquée.

In [ ]:
# --- Dictionnaire de correspondance pour les couleurs ---

color_mapping = {
    "bleu foncé": "darkblue",
    "blanc bleuté": "lightblue",
    "bleu moyen": "mediumblue",
    "bleu turquoise": "turquoise",
    "vert": "green",
    "rouge foncé": "darkred",
    "violet": "purple",
    "rouge clair": "lightcoral",
    "rose": "pink",
    "blanc": "grey",
    "jaune": "yellow",
    "brun": "black" 
}
reliure_pca_df["ColorMapped"] = reliure_pca_df["Couleur"].map(color_mapping)

# Récupérer automatiquement les labels pour Dim1, Dim2 et Dim3
dim1_label = f"Dim1 ({eig.loc[eig['Dimension']=='Dim1', '% variance expliquée'].values[0]}%)"
dim2_label = f"Dim2 ({eig.loc[eig['Dimension']=='Dim2', '% variance expliquée'].values[0]}%)"
dim3_label = f"Dim3 ({eig.loc[eig['Dimension']=='Dim3', '% variance expliquée'].values[0]}%)"

# --- Création de la grille 3 lignes x 2 colonnes pour les 6 graphiques ---
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(15, 20))

# ---------- Ligne 1 : Dim1 vs Dim2 ----------
# Gauche : sans annotations
axs[0, 0].scatter(reliure_pca_df["Dim1"], reliure_pca_df["Dim2"],
                  c=reliure_pca_df["ColorMapped"], s=100)
axs[0, 0].set_xlabel(dim1_label, fontsize=12)
axs[0, 0].set_ylabel(dim2_label, fontsize=12)
axs[0, 0].set_title("Dim1 vs Dim2", fontsize=14)
axs[0, 0].grid(True)

# Droite : avec annotations
axs[0, 1].scatter(reliure_pca_df["Dim1"], reliure_pca_df["Dim2"],
                  c=reliure_pca_df["ColorMapped"], s=100)
for index, row in reliure_pca_df.iterrows():
    # Vous pouvez adapter la condition d'annotation selon vos critères
    if (abs(row["Dim1"]) > 5) or (abs(row["Dim2"]) > 0):
        axs[0, 1].annotate(row["Localisation"],
                           (row["Dim1"], row["Dim2"]),
                           fontsize=9,
                           textcoords="offset points",
                           xytext=(5, 5))
axs[0, 1].set_xlabel(dim1_label, fontsize=12)
axs[0, 1].set_ylabel(dim2_label, fontsize=12)
axs[0, 1].set_title("Dim1 vs Dim2 avec annotations", fontsize=14)
axs[0, 1].grid(True)

# ---------- Ligne 2 : Dim1 vs Dim3 ----------
# Gauche : sans annotations
axs[1, 0].scatter(reliure_pca_df["Dim1"], reliure_pca_df["Dim3"],
                  c=reliure_pca_df["ColorMapped"], s=100)
axs[1, 0].set_xlabel(dim1_label, fontsize=12)
axs[1, 0].set_ylabel(dim3_label, fontsize=12)
axs[1, 0].set_title("Dim1 vs Dim3", fontsize=14)
axs[1, 0].grid(True)

# Droite : avec annotations
axs[1, 1].scatter(reliure_pca_df["Dim1"], reliure_pca_df["Dim3"],
                  c=reliure_pca_df["ColorMapped"], s=100)
for index, row in reliure_pca_df.iterrows():
    if (abs(row["Dim1"]) > 5) or (abs(row["Dim3"]) > 0):
        axs[1, 1].annotate(row["Localisation"],
                           (row["Dim1"], row["Dim3"]),
                           fontsize=9,
                           textcoords="offset points",
                           xytext=(5, 5))
axs[1, 1].set_xlabel(dim1_label, fontsize=12)
axs[1, 1].set_ylabel(dim3_label, fontsize=12)
axs[1, 1].set_title("Dim1 vs Dim3 avec annotations", fontsize=14)
axs[1, 1].grid(True)

# ---------- Ligne 3 : Dim2 vs Dim3 ----------
# Gauche : sans annotations
axs[2, 0].scatter(reliure_pca_df["Dim2"], reliure_pca_df["Dim3"],
                  c=reliure_pca_df["ColorMapped"], s=100)
axs[2, 0].set_xlabel(dim2_label, fontsize=12)
axs[2, 0].set_ylabel(dim3_label, fontsize=12)
axs[2, 0].set_title("Dim2 vs Dim3", fontsize=14)
axs[2, 0].grid(True)

# Droite : avec annotations
axs[2, 1].scatter(reliure_pca_df["Dim2"], reliure_pca_df["Dim3"],
                  c=reliure_pca_df["ColorMapped"], s=100)
for index, row in reliure_pca_df.iterrows():
    if (abs(row["Dim2"]) > 5) or (abs(row["Dim3"]) > 0):
        axs[2, 1].annotate(row["Localisation"],
                           (row["Dim2"], row["Dim3"]),
                           fontsize=9,
                           textcoords="offset points",
                           xytext=(5, 5))
axs[2, 1].set_xlabel(dim2_label, fontsize=12)
axs[2, 1].set_ylabel(dim3_label, fontsize=12)
axs[2, 1].set_title("Dim2 vs Dim3 avec annotations", fontsize=14)
axs[2, 1].grid(True)

plt.tight_layout()
# On enregistre l'image en .jpeg
plt.savefig("pca_plots.jpeg", format="jpeg", dpi=300, bbox_inches="tight")
plt.show()



## Représentation en 3D 

In [ ]:
# --- Création de la grille 1 lignes x 2 colonnes pour les 2 graphiques ---
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 20), subplot_kw={'projection': "3d"})


# Gauche : sans annotations
axs[0].scatter(reliure_pca_df["Dim1"], reliure_pca_df["Dim2"], reliure_pca_df["Dim3"],
                  c=reliure_pca_df["ColorMapped"], s=100)
axs[0].set_xlabel(dim1_label, fontsize=12)
axs[0].set_ylabel(dim2_label, fontsize=12)
axs[0].set_zlabel(dim3_label, fontsize=12)
axs[0].grid(True)

# Droite : avec annotations - ça ne marche pas, à modifier plus tard
axs[1].scatter(reliure_pca_df["Dim1"], reliure_pca_df["Dim2"], reliure_pca_df["Dim3"],
                  c=reliure_pca_df["ColorMapped"], s=100)
for index, row in reliure_pca_df.iterrows():
    # Vous pouvez adapter la condition d'annotation selon vos critères
    if (abs(row["Dim1"]) > 5) or (abs(row["Dim2"]) > 0):
        axs[1].annotate(row["Localisation"],
                           (row["Dim1"], row["Dim2"]),
                           fontsize=9,
                           textcoords="offset points",
                           xytext=(5, 5))
axs[1].set_xlabel(dim1_label, fontsize=12)
axs[1].set_ylabel(dim2_label, fontsize=12)
axs[0].set_zlabel(dim3_label, fontsize=12)
axs[1].grid(True)

plt.tight_layout()
plt.show()


## Représentation des variables 

In [ ]:
n=reliure_num.shape[0] # nombre d'individus
p=reliure_num.shape[1] # nombre de variables
n
eigval = (n-1)/n*pca.explained_variance_ #valeurs propres
sqrt_eigval = np.sqrt(eigval) # racine carré des valeurs propres
n_components = pca.components_.shape[0]  # ou simplement len(sqrt_eigval)

corvar = np.zeros((p, n_components)) # attention à l'orientation
for k in range(n_components):
    corvar[:,k] = pca.components_[k,:] * sqrt_eigval[k]
# on modifie pour avoir un dataframe
coordvar = pd.DataFrame({'id': reliure_num.columns, 'COR_1': corvar[:,0], 'COR_2': corvar[:,1]})
coordvar.head(3)

On peut ensuite afficher le cercle des corrélations

In [ ]:
# Création d'une figure vide (avec des axes entre -1 et 1 + le titre)
fig, axes = plt.subplots(figsize = (6,6))
fig.suptitle("Cercle des corrélations")
axes.set_xlim(-1, 1)
axes.set_ylim(-1, 1)
# Ajout des axes
axes.axvline(x = 0, color = 'lightgray', linestyle = '--', linewidth = 1)
axes.axhline(y = 0, color = 'lightgray', linestyle = '--', linewidth = 1)
# Ajout des noms des variables
for j in range(p):
    axes.text(coordvar["COR_1"][j],coordvar["COR_2"][j], coordvar["id"][j])
# Ajout du cercle
plt.gca().add_artist(plt.Circle((0,0),1,color='blue',fill=False))
plt.show()



## Représentation conjointe des loadings et des individus

In [ ]:
# Récupérer la liste des éléments chimiques (les variables utilisées dans la PCA)
chemical_elements = reliure_num.columns

# Calculer les loadings : pca.components_ est de forme (n_components, n_features)
# On transpose pour obtenir un tableau de forme (n_features, n_components)
loadings = pca.components_.T

# Construction automatique des labels des axes à partir de eig
dim1_label = f"Dim1 ({eig.loc[eig['Dimension']=='Dim1', '% variance expliquée'].values[0]}%)"
dim2_label = f"Dim2 ({eig.loc[eig['Dimension']=='Dim2', '% variance expliquée'].values[0]}%)"
dim3_label = f"Dim3 ({eig.loc[eig['Dimension']=='Dim3', '% variance expliquée'].values[0]}%)"

# Fonction pour ajouter les flèches (loadings) avec un décalage pour éviter les chevauchements
def add_loadings(ax, dim_x, dim_y, scale=3, offset=0.2):
    """
    Ajoute sur l'axe ax les flèches et étiquettes des éléments chimiques pour les dimensions d'indice dim_x et dim_y.
    Le paramètre scale ajuste la longueur des flèches, et offset permet de décaler les textes.
    """
    for i, var in enumerate(chemical_elements):
        x_arrow = loadings[i, dim_x] * scale
        y_arrow = loadings[i, dim_y] * scale
        ax.arrow(0, 0, x_arrow, y_arrow, color='black', width=0.005, head_width=0.1)
        # On déplace l'étiquette en multipliant par (1 + offset)
        ax.text(x_arrow * (1 + offset), y_arrow * (1 + offset), var,
                color='black', fontsize=10, ha='center', va='center')

# Création de la grille 2x2 (4 sous-graphiques)
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(16, 14))

# ---------- Diagramme 1 (Haut-gauche): Dim1 vs Dim2 ----------
ax1 = axs[0,0]
ax1.scatter(reliure_pca_df["Dim1"], reliure_pca_df["Dim2"],
            c=reliure_pca_df["ColorMapped"], s=100, alpha=0.7)
# Annotation des individus (par exemple, avec "Localisation")
for index, row in reliure_pca_df.iterrows():
    ax1.annotate(row["Localisation"],
                 (row["Dim1"], row["Dim2"]),
                 fontsize=9,
                 textcoords="offset points",
                 xytext=(5, 5))
ax1.set_xlabel(dim1_label, fontsize=12)
ax1.set_ylabel(dim2_label, fontsize=12)
ax1.set_title("Dim1 vs Dim2", fontsize=14)
ax1.grid(True)
add_loadings(ax1, dim_x=0, dim_y=1, scale=3, offset=0.2)

# ---------- Diagramme 2 (Haut-droite): Dim1 vs Dim3 ----------
ax2 = axs[0,1]
ax2.scatter(reliure_pca_df["Dim1"], reliure_pca_df["Dim3"],
            c=reliure_pca_df["ColorMapped"], s=100, alpha=0.7)
for index, row in reliure_pca_df.iterrows():
    ax2.annotate(row["Localisation"],
                 (row["Dim1"], row["Dim3"]),
                 fontsize=9,
                 textcoords="offset points",
                 xytext=(5, 5))
ax2.set_xlabel(dim1_label, fontsize=12)
ax2.set_ylabel(dim3_label, fontsize=12)
ax2.set_title("Dim1 vs Dim3", fontsize=14)
ax2.grid(True)
add_loadings(ax2, dim_x=0, dim_y=2, scale=3, offset=0.2)

# ---------- Diagramme 3 (Bas-gauche): Dim2 vs Dim3 ----------
ax3 = axs[1,0]
ax3.scatter(reliure_pca_df["Dim2"], reliure_pca_df["Dim3"],
            c=reliure_pca_df["ColorMapped"], s=100, alpha=0.7)
for index, row in reliure_pca_df.iterrows():
    ax3.annotate(row["Localisation"],
                 (row["Dim2"], row["Dim3"]),
                 fontsize=9,
                 textcoords="offset points",
                 xytext=(5, 5))
ax3.set_xlabel(dim2_label, fontsize=12)
ax3.set_ylabel(dim3_label, fontsize=12)
ax3.set_title("Dim2 vs Dim3", fontsize=14)
ax3.grid(True)
add_loadings(ax3, dim_x=1, dim_y=2, scale=3, offset=0.2)

# ---------- Panneau 4 (Bas-droite): vide ou à utiliser pour une légende par exemple ----------
axs[1,1].axis("off")

plt.tight_layout()
plt.savefig("pca_plots_variables.jpeg", format="jpeg", dpi=300, bbox_inches="tight")
plt.show()


## Classification k-means

Nous allons l'appliquer avec 2 et 3 classes afin de voir quelle partition serait la plus intéressante.
On peut avoir ainsi les classes de chaque individus (qui nous servent ici à calculer la taille de chaque classe), ainsi que les centres des classes.
L'algorithme k-means nous permet d'avoir à la fin l'inertie intra-classes, qui représente la disparité des individus à l'intérieur des classes. Plus cette valeur est proche de 0, meilleur est la partition. Malheureusement, la meilleure partition selon ce critère est donc celle avec autant de classes que d'individus. On va donc chercher un point d'inflexion dans la courbe d'évolution de ce critère. Voici comment faire pour avoir ce graphique. Et ici, le point le plus marquant est celui à 2 classes. Ensuite, celui à 3 classes peut montrer aussi une certaine cassure dans l'évolution du critère.

In [ ]:
from sklearn.cluster import KMeans
kmeans2 = KMeans(n_clusters = 2)
kmeans2.fit(reliure_num)
print(pd.Series(kmeans2.labels_).value_counts())
#kmeans2.cluster_centers_
reliure_k2 = reliure_num.assign(classe = kmeans2.labels_)
reliure_k2.groupby("classe").mean()

In [ ]:
kmeans3 = KMeans(n_clusters = 3)
kmeans3.fit(reliure_num)
print(pd.Series(kmeans3.labels_).value_counts())
reliure_k3 = reliure_num.assign(classe = kmeans3.labels_)
reliure_k3.groupby("classe").mean()

In [ ]:
# --- Création de la grille 2x2 ---
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(15, 15))

# ---------- Top-Left : Clustering k-means avec 2 groupes ----------
kmeans2 = KMeans(n_clusters=2, random_state=42)
kmeans2.fit(reliure_num)
# On assigne les labels au DataFrame PCA
reliure_pca_k2 = reliure_pca_df.assign(classe=kmeans2.labels_)
axs[0, 0].scatter(reliure_pca_k2["Dim1"], reliure_pca_k2["Dim2"],
                  c=reliure_pca_k2["classe"], cmap="Set1", s=100)
axs[0, 0].set_xlabel(dim1_label, fontsize=12)
axs[0, 0].set_ylabel(dim2_label, fontsize=12)
axs[0, 0].set_title("Clustering k-means = 2", fontsize=14)
axs[0, 0].grid(True)

# ---------- Top-Right : Clustering k-means avec 3 groupes ----------
kmeans3 = KMeans(n_clusters=3, random_state=42)
kmeans3.fit(reliure_num)
reliure_pca_k3 = reliure_pca_df.assign(classe=kmeans3.labels_)
axs[0, 1].scatter(reliure_pca_k3["Dim1"], reliure_pca_k3["Dim2"],
                  c=reliure_pca_k3["classe"], cmap="Set1", s=100)
axs[0, 1].set_xlabel(dim1_label, fontsize=12)
axs[0, 1].set_ylabel(dim2_label, fontsize=12)
axs[0, 1].set_title("Clustering k-means = 3", fontsize=14)
axs[0, 1].grid(True)

# ---------- Bottom-Left : Courbe d'inertie pour k = 1 à 10 ----------
inertia = []
for k in range(1, 11):
    km = KMeans(n_clusters=k, init="random", n_init=20, random_state=42).fit(reliure_num)
    inertia.append(km.inertia_)
inertia_df = pd.DataFrame({"k": range(1, 11), "inertia": inertia})
axs[1, 0].plot(inertia_df["k"], inertia_df["inertia"], marker="o", linestyle="-")
# Mettre en évidence k = 2 et k = 3
axs[1, 0].scatter(2, inertia_df[inertia_df["k"] == 2]["inertia"], c="red", s=100)
axs[1, 0].scatter(3, inertia_df[inertia_df["k"] == 3]["inertia"], c="red", s=100)
axs[1, 0].set_xlabel("k", fontsize=12)
axs[1, 0].set_ylabel("Inertia", fontsize=12)
axs[1, 0].set_title("Courbe d'inertie k-means", fontsize=14)
axs[1, 0].grid(True)

# ---------- Bottom-Right : Panneau vide ----------
axs[1, 1].axis("off")

plt.tight_layout()
plt.show()